<a href="https://colab.research.google.com/github/romenlaw/RL-playground/blob/main/rl_playground4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

GPRL book chapter 8 - Introduction to Value-Based Deep Reinforcement Learning

# Preparation

In [205]:
import warnings ; warnings.filterwarnings('ignore')
import os

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import numpy as np
from IPython.display import display
from collections import namedtuple, deque
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
from itertools import cycle, count
from textwrap import wrap

import matplotlib
import subprocess
import os.path
import tempfile
import random
import base64
import pprint
import glob
import time
import json
import sys
import gym
import io
import os
import gc

from gym import wrappers
from subprocess import check_output
from IPython.display import HTML

LEAVE_PRINT_EVERY_N_SECS = 60
ERASE_LINE = '\x1b[2K'
EPS = 1e-6
BEEP = lambda: os.system("printf '\a'")
RESULTS_DIR = os.path.join('..', 'results')
SEEDS = (12, 34, 56, 78, 90)

%matplotlib inline

In [206]:
plt.style.use('fivethirtyeight')
params = {
  'figure.figsize': (15, 8),
  'font.size': 24,
  'legend.fontsize': 20,
  'axes.titlesize': 28,
  'axes.labelsize': 24,
  'xtick.labelsize': 20,
  'ytick.labelsize': 20
}
pylab.rcParams.update(params)
np.set_printoptions(suppress=True)

In [207]:
torch.cuda.is_available()

False

# Utils

In [208]:
def get_make_env_fn(**kargs):
  def make_env_fn(env_name,
                  seed=None,
                  render=None,
                  record=False,
                  unwrapped=False,
                  monitor_mode=None, # record video
                  inner_wrappers=None,
                  outer_wrappers=None):
    tmp_dir = tempfile.mkdtemp()
    env = None
    if render:
      try:
        env = gym.make(env_name, render=render)
      except:
        pass
    if env is None: env = gym.make(env_name)
    if seed is not None: env.seed(seed)
    env = env.unwrapped if unwrapped else env
    if inner_wrappers:
      for wrapper in inner_wrappers:
        env = wrapper(env)

    env = wrappers.Monitor(env, tmp_dir, force=True, # override temp file
                           mode=monitor_mode,
                           video_callable=lambda e_idx: record
                           ) if monitor_mode else env
    if outer_wrappers:
      for wrapper in outer_wrappers:
        env = wrapper(env)
    return env
  return make_env_fn, kargs

In [209]:
def get_videws_html(env_videos, title, max_n_videos=5):
  '''
  env_videos - list of videos
  title - heading/title of the html heading
  max_n_videos - max number of videos; can be larger or smaller tha len of the env_videos list
  '''
  videos = np.array(env_videos)
  if len(videos) ==0:
    return
  n_videos = max(1, min(max_n_videos, len(videos)))
  idxs = np.linspace(0, len(videos)-1, n_videos).astype(int) if n_videos>1 else [-1,]
  videos = videos[idxs, ...]

  strm = f'<h2>{title}</h2>'
  for video_path, meta_path in videos:
    video = io.open(video_path, 'r+b').read()
    encoded = base64.b64encode(video)

    with open(meta_path) as data_file:
      meta = json.load(data_file)

    html_tag=f"""
    <h3>Episode {meta['episode_id']}</h3>
    <video width="960" height="540" controls>
      <source src="data:video/mp4;base64,{encoded.decode('ascii')}" type="video/mp4"/>
    </video>
    """
    strm += html_tag
  return strm


In [210]:
def get_gif_html(env_videos, title, subtitle_eps=None, max_n_videos=4):
  videos = np.array(env_videos)
  if len(videos) ==0:
    return
  n_videos = max(1, min(max_n_videos, len(videos)))
  idxs = np.linspace(0, len(videos)-1, n_videos).astype(int) if n_videos>1 else [-1,]
  videos = videos[idxs, ...]

  strm = f'<h2>{title}</h2>'
  for video_path, meta_path in videos:
    basename = os.path.splitext(video_path)[0]
    gif_path = basename + '.gif'
    if not os.path.exists(gif_path):
      ps = subprocess.Popen((
          'ffmpeg',
          '-i', video_path,
          '-r', '7',
          '-f', 'image2pipe',
          '-vcodec', 'ppm',
          '-crf', '20',
          '-vf', 'scale=512:-1',
          '-'
      ), stdout=subprocess.PIPE)
      output = subprocess.check_output(
          ('convert',
           '-coalesce',
           '-delay', '7',
           '-loop', '0',
           '-fuzz', '2%',
           '+dither',
           '-deconstruct',
           '-layers', 'Optimize',
           '-', gif_path),
          stdin=ps.stdout)
      ps.wait()

    gif = io.open(gif_path, 'r+b').read()
    encoded = base64.b64encode(gif)

    with open(meta_path) as data_file:
      meta = json.load(data_file)

    prefix = 'Trial ' if subtitle_eps is None else 'Episode '
    sufix = meta['episode_id'] if subtitle_eps is None else subtitle_eps[meta['episode_id']]
    html_tag = f"""
    <h3>{prefix}{sufix}</h3>
    <img src="data:image/gif;base64,{encoded.decode('ascii')}"/>
    """
    strm += html_tag
  return strm

In [211]:
class DiscountedCartPole(gym.Wrapper):
  def __init__(self, env):
    gym.Wrapper.__init__(self, env)

  def reset(self, **kwargs):
    return self.env.reset(**kwargs)

  def setp(self, a):
    # step returns observation, reward, terminated, truncated, info
    o, r, d, _ = self.env.step(a)
    (x, x_dot, theta, theta_dot) = o
    pole_fell = x < -self.env.unwrapped.x_threshold \
      or x > self.env.unwrapped.x_threshold \
      or theta < -self.env.unwrapped.theta_threshold_radians \
      or theta > self.env.unwrapped.theta_threshold_radians
    r = -1 if pole_fell else 0
    return o, r, d, _

# NFQ
Neural Fitted Q Iteration (NFQ), one of the 1st neuro-net algo for reinforcment learning. It approximates action-value function - estimated as Q(s,a; θ).

The architecture we use for cart pole is State-in-values-out:
* input = state variables s: cart position, velocity, pole angle, velocity at tip
* output = vector of values Q(s): one for each action

In [212]:
# fully connected Q-function (state-in-values-out)
class FCQ(nn.Module):
  def __init__(self, input_dim, output_dim, hidden_dims=(32,32), activation_fc=F.relu):
    super(FCQ, self).__init__()
    self.activation_fc = activation_fc

    self.input_layer = nn.Linear(input_dim, hidden_dims[0])
    self.hidden_layers = nn.ModuleList()
    for i in range(len(hidden_dims)-1):
      hidden_layer = nn.Linear(hidden_dims[i], hidden_dims[i+1])
      self.hidden_layers.append(hidden_layer)

    self.output_layer = nn.Linear(hidden_dims[-1], output_dim)

    device = "cuda:0" if torch.cuda.is_available() else "cpu"
    self.device = torch.device(device)
    self.to(self.device)

  def _format(self, state):
    """
    state: is a list of state variables, cart position, velocity, etc.
    """
    x = state
    if not isinstance(x, torch.Tensor):
      x = torch.tensor(x, device=self.device, dtype=torch.float32)
      x = x.unsqueeze(0)
    return x

  def forward(self, state):
    x = self._format(state)
    x = self.activation_fc(self.input_layer(x))
    for hidden_layer in self.hidden_layers:
      x = self.activation_fc(hidden_layer(x))
    x = self.output_layer(x)
    return x

  def numpy_float_to_device(self, variable):
    variable = torch.from_numpy(variable).float().to(self.device)
    return variable

  def load(self, experiences):
    states, actions, rewards, new_states, is_terminals = experiences
    states = torch.from_numpy(states).float().to(self.device)
    actions = torch.from_numpy(actions).long().to(self.device)
    new_states = torch.from_numpy(new_states).float().to(self.device)
    rewards = torch.from_numpy(rewards).float().to(self.device)
    is_termnials = torch.from_numpy(is_terminals).float().to(self.device)
    return states, actions, rewards, new_states, is_terminals


## selecting an exploration strategy

Any techniques from chapter 4 for exploration-exploitation tradeoff will be fine. To keep it simple, we use an epsilon-greedy strategy on our NFQ implementation.

But we are trining an off-policy learning algo, i.e. there are two policies:
* a policy that generates behaviour, which in this case is an ϵ-greedy policy
* a policy that we are learning about, which is the greedy (an ultimately optimal) policy.

In [213]:
class GreedyStrategy():
  def __init__(self):
    self.exploratory_action_taken = False

  def select_action(self, model, state):
    with torch.no_grad():
      q_values = model(state).cpu().detach().data.numpy().squeeze()
      return np.argmax(q_values)

In [214]:
class EGreedyStrategy():
  def __init__(self, epsilon=0.1):
    self.epsilon = epsilon
    self.exploratory_action_taken = None

  def select_action(self, model, state):
    self.exploratory_action_taken = False
    with torch.no_grad():
      q_values = model(state).cpu().detach().data.numpy().squeeze()

    if np.random.rand() > self.epsilon:
      action = np.argmax(q_values)
    else:
      action = np.random.randint(len(q_values))

    self.exploratory_action_taken = action != np.argmax(q_values)
    return action

## selecting what to optimise
An ideal objective in value-based DRL would be to minimise the loss w.r.t. the optimal action-value function q⋆.
$$L_i(θ_i)=\mathbb{E}_{s,a}\left[\left(q_*(s,a)-Q(s,a;\theta_i)\right)^2\right]
$$
Because we'd like to have an estimate of q*, Q, that tracks exactly that optimal function, if we had a solid estimate of q*, we then could use a greedy action w.r.t. these estimates to get near optimal behaviour.

Refesh the definition of optimal action=value function q⋆:
$$q_*(s,a)=\max_{π}\mathbb{E}_π\left[G_t|S_t=s, A_t=a\right], ∀s∈S,∀a\in A(s)
$$
The optimal action-value function q⋆ is the poicy π that gives the maximum expected return from each and every action in each and every state.

For our NFQ implementation, we start with a randomly initialised q function (and implicit policy). Then evaluate the policy by sampling actions from it, same as chapter 5. Then iprove it with an exploration strategy such as epsilon-greedy, like in chapter 4. Finally, keep iterating until we reach the desired peformance, as in chapters 6 and 7.

## selecting targets for policy evaluation
Out of MC, TD, N-step, Lambda targets, any one would do. For simplicity we choose TD target.

TD targets can be either
* on-policy: Sarsa target for on-policy bootstrapping - use the action-value function of the action the agent will take at the landing state; the target would be approximatin gthe behavioural policy, i.e. the policy-generating behaviour and the policy being learned would be the same.
* off-policy: Q-learning for off-policy bootstrapping - use the value of the best action at the next state; we always approximate the greedy poicy, even if the policy-generating behaviour is not totally greedy.
$$
y_{i}^{Sarsa}=R_{t+1}+γQ(S_{t+1},A_{t+1};θ_i)
$$
$$
y_{i}^{Q-learning}=R_{t+1}+γ\max_{a}Q(S_{t+1},a;θ_i)
$$

In our NFQ implemenation, we will use the off-policy TD target. Substituting $y_i^{Q-learning}$ into the above loss function $L_i$:
$$L_i(θ_i)=\mathbb{E}_{s,a,r,s'}\left[\left(r+γ\max_{a'}Q(s',a';θ_i)-Q(s,a;\theta_i)\right)^2\right]
$$

Notice that when differentiating througth this equation, the gradiet does not involve the target y, the gradient must only go through the predicted value Q:
$$
∇_{θ_i} L_i(θ_i)=\mathbb{E}_{s,a,r,s'}\left[\left(r+γ\max_{a'}Q(s',a';θ_i)-Q(s,a;\theta_i)\right)\nabla_{θ_i}Q(s,a;\theta_i)\right]
$$

In [215]:
class NFQ():
  def __init__(self,
               value_model_fn,
               value_optimizer_fn,
               value_optimizer_lr,
               training_strategy_fn,
               evaluation_strategy_fn,
               batch_size,
               epochs):
    self.value_model_fn = value_model_fn
    self.value_optimizer_fn = value_optimizer_fn
    self.value_optimizer_lr = value_optimizer_lr
    self.training_strategy_fn = training_strategy_fn
    self.evaluation_strategy_fn = evaluation_strategy_fn
    self.batch_size = batch_size
    self.epochs = epochs

  def optimize_model(self, experiences):
    states, actions, rewards, next_states, is_terminals = experiences
    batch_size = len(is_terminals)

    # we get the values of Q-function at s_prime (next_state).
    # 'next_states' is plural because it's a batch of 'next_state'.
    # detach() here becasue we don't propagate values through this, this keeps
    # the predicted vlaue a constant since we only use it to calculate targets.
    q_sa = self.online_model(next_states).detach()
    max_a_q_sp = q_sa.max(1)[0].unsqueeze(1)
    # *(1-is_terminals) to ensure terminal states are grounded to zero.
    target_q_s = rewards + self.gamma * max_a_q_sp * (1 - is_terminals)

    # get the current estimate of Q(s,a).
    q_sa = self.online_model(states).gather(1, actions)

    td_errors = target_q_s - q_sa
    value_loss = td_errors.pow(2).mul(0.5).mean()
    # back prop
    self.value_optimizer.zero_grad()
    value_loss.backward()
    self.value_optimizer.step()

  def interaction_step(self, state, env):
    action = self.training_strategy.select_action(self.online_model, state)
    new_state, reward, is_terminal, info = env.step(action)
    # the cart-pole env has a wrapper that artificially teminates an episode after
    # some time steps. So we add extra code to check this.
    is_truncated = 'TimeLimit.truncated' in info and info['TimeLimit.truncated']
    is_failure = is_terminal and not is_truncated
    experience = (state, action, reward, new_state, float(is_failure))
    # print(state)
    # print(action)
    # print(reward)
    # print(new_state)
    # print(experience) ###############
    # print('============')

    self.experiences.append(experience)
    self.episode_reward[-1] += reward
    self.episode_timestep[-1] += 1
    self.episode_exploration[-1] += int(self.training_strategy.exploratory_action_taken)
    return new_state, is_terminal

  def train(self, make_env_fn, make_env_kargs,
            seed, gamma,
            max_minutes, max_episodes, goal_mean_100_reward):
    training_start, last_debug_time = time.time(), float('-inf')

    self.checkpoint_dir = tempfile.mkdtemp()
    self.make_env_fn = make_env_fn
    self.make_env_kargs = make_env_kargs
    self.seed = seed
    self.gamma = gamma

    env = self.make_env_fn(**self.make_env_kargs, seed=self.seed)
    torch.manual_seed(self.seed); np.random.seed(self.seed); random.seed(self.seed)

    nS, nA = env.observation_space.shape[0], env.action_space.n
    self.episode_timestep = []
    self.episode_reward = []
    self.episode_seconds = []
    self.evaluation_scores = []
    self.episode_exploration = []

    self.online_model = self.value_model_fn(nS, nA)
    self.value_optimizer = self.value_optimizer_fn(self.online_model,
                                                   self.value_optimizer_lr)

    self.training_strategy = training_strategy_fn()
    self.evaluation_strategy = evaluation_strategy_fn()
    self.experiences = []

    # the 5 dim are: total_step, mean_100_reward, mean_100_eval_score, training_time, wallclock_elapsed
    result = np.empty((max_episodes, 5))
    result[:] = np.nan
    training_time = 0
    for episode in range(1, max_episodes+1):
      episode_start = time.time()
      state, is_terminal = env.reset(), False
      self.episode_reward.append(0.0)
      self.episode_timestep.append(0.0)
      self.episode_exploration.append(0.0)

      for step in count(): # starting from 0
        state, is_treminal = self.interaction_step(state, env)
        # self.experiences is appended by self.interaction_step()

        if len(self.experiences) >= self.batch_size:
          experiences = np.array(self.experiences, dtype=object)
          batches = [np.vstack(sars) for sars in experiences.T]
          experiences = self.online_model.load(batches)
          for _ in range(self.epochs):
            self.optimize_model(experiences)
          self.experiences.clear()

        if is_terminal:
          gc.collect()
          break

      # stats
      episode_elapsed = time.time() - episode_start
      self.episode_seconds.append(episode_elapsed)
      training_time += episode_elapsed
      evaluation_score, _ = self.evaluate(self.online_model, env)
      self.save_checkpoint(episode-1, self.online_model)

      total_step = int(np.sum(self.episode_timestep))
      self.evaluation_scores.append(evaluation_score)

      mean_10_reward = np.mean(self.episode_reward[-10:])
      std_10_reward = np.std(self.episode_reward[-10:])

      mean_100_reward = np.mean(self.episode_reward[-100:])
      std_100_reward = np.std(self.episode_reward[-100:])

      mean_100_eval_score = np.mean(self.evaluation_scores[-100:])
      std_100_eval_score = np.std(self.evaluation_scores[-100:])

      lst_100_exp_rat = np.array(self.episode_exploration[-100:]) \
        / np.array(self.episode_timestep[-100:])
      mean_100_exp_rat = np.mean(lst_100_exp_rat)
      std_100_exp_rat = np.std(lst_100_exp_rat)

      wallclock_elapsed = time.time() - training_start
      result[episode-1] = total_step, mean_100_reward, \
        mean_100_eval_score, training_time, wallclock_elapsed

      reached_debug_time = time.time() - last_debug_time >= LEAVE_PRINT_EVERY_N_SECS
      reached_max_minutes = wallclock_elapsed >= max_minutes * 60
      reached_max_episodes = episode >= max_episodes
      reached_goal_mean_reward = mean_100_eval_score >= goal_mean_100_reward
      training_is_over = reached_max_minutes or reached_max_episodes \
        or reached_goal_mean_reward

      elapsed_str = time.strftime("%H:%M:%S", time.gmtime(time.time()-training_start))
      debug_message = 'el {}, ep {:04}, ts {:06}, ' \
        'ar 10 {:05.1f}\u00B1{:05.1f}, ' \
        '100 {:05.1f}\u00B1{:05.1f}, ' \
        'ex 100 {:02.1f}\u00B1{:02.1f}, ' \
        'ev {:05.1f}\u00B1{:05.1f}'
      debug_message = debug_message.format(
        elapsed_str, episode-1, total_step, mean_10_reward, std_10_reward,
        mean_100_reward, std_100_reward, mean_100_exp_rat, std_100_exp_rat,
        mean_100_eval_score, std_100_eval_score
      )
      print(debug_message, end='\r', flush=True)
      if reached_debug_time or training_is_over:
        print(ERASE_LINE + debug_message, flush=True)
        last_debug_time = time.time()
      if training_is_over: # print crosses and ticks
        if reached_max_minutes: print(u'--> reached_max_minutes \u2715')
        if reached_max_episodes: print(u'--> reached_max_episodes \u2715')
        if reached_goal_mean_reward: print(u'--> reach_goal_mean_reward \u2713')
        break # break out of for step in count() loop

    final_eval_score, score_std = self.evaluate(self.online_model, env, n_episodes=100)
    wallclock_time = time.time() - training_start
    print('Training complete.')
    print(f'Final evaluation score {final_eval_score:.2f}\u00B1{score_std:.2f}'\
          ' in {training_time:.2f}s training time, {wallclock_time:.2f}s wallclock time.\n')
    env.close(); del env
    self.get_cleaned_checkpoints()
    return result, final_eval_score, training_time, wallclock_time

  def evaluate(self, eval_policy_model, eval_env, n_episodes=1):
    rs = [] # rewards
    for _ in range(n_episodes):
      # state, is_terminal
      s, d = eval_env.reset(), False
      rs.append(0)
      for _ in count():
        a = self.evaluation_strategy.select_action(eval_policy_model, s)
        s, r, d, _ = eval_env.step(a)
        rs[-1] += r
        if d: break
    return np.mean(rs), np.std(rs)

  def get_cleaned_checkpoints(self, n_checkpoints=5):
    try:
      return self.checkpoint_paths
    except AttributeError:
      self.checkpint_paths={}

    paths = glob.glob(os.path.join(self.checkpoint_dir, '*.tar'))
    # the file name is like 'dir/model.0.tar', 'dir/model.1.tar', etc.
    paths_dic = {int(path.split('.')[-2]):path for path in paths}
    last_ep = max(paths_dic.keys())
    checkpoint_idxs = np.linespace(1, last_ep+1, n_checkpoints, endpoint=True, dtype=np.int16)-1

    for idx, path in paths_dic.items():
      if idx in checkpoint_idxs:
        self.checkpoint_paths[idx] = path
      else:
        os.unlink(path)

    return self.ceckpoint_paths

  def demo_last(self, title='Fully-trained {} Agent', n_episodes=3, max_n_videos=3):
    env = self.make_env_fn(**self.make_env_kargs, monitor_mode='evaluation', render=True, record=True)

    checkpoint_paths = self.get_cleaned_checkpoints()
    last_ep = max(checkpoint_paths.keys())
    self.online_model.load_state_dict(torch.load(checkpoint_paths[last_ep]))

    self.evaluate(self.online_model, env, n_episodes=n_episodes)
    env.close()
    data = get_gif_html(env_videos=env.videos,
                        title=title.format(self.__class__.__name__),
                        max_n_videws=max_n_videos)
    del env
    return HTML(data=data)

  def demo_progression(self, title='{} Agent progression', max_n_videos=5):
    env = self.make_env_fn(**self.make_env_kargs, monitor_mode='evaluation', render=True, record=True)

    checkpoint_paths = self.get_cleaned_checkpoints()
    for i in sorted(checkpoint_paths.keys()):
      self.online_model.load_state_dict(torch.load(checkpoint_paths[i]))
      self.evaluate(self.online_model, env, n_episodes=1)

    env.close()
    data = get_gif_html(env_videos=env.videos,
                        title=title.format(self.__class__.__name__),
                        subtitle_eps=sorted(checkpoint_paths.keys()),
                        max_n_videos=max_n_videos)
    del env
    return HTML(data=data)

  def save_checkpoint(self, episode_idx, model):
    torch.save(model.state_dict(),
               os.path.join(self.checkpoint_dir, f'model.{episode_idx}.tar'))

## select an Optimization method
Gradient descent is a stable optimisation method given the following assumptions:
1. IID assumption - Idependent and Identically Distributed.
2. targets are stationary.

However, in RL, the above assumptions don't hold. In practice, both Adam (Adaptive movement estimation) and RMSprop are sensible choices for value-based DRL methods.

NFQ has 3 main steps:
1. collect E experiences: (s, a, r, s', d) tuples. We use 1024 samples. This is batch_size in code
2. calculate the off-policy TD targets: $r+γ\max_{a'}Q(s',a';θ)$
3. fit the action-value function Q(s,a;θ): using MSE and RMSprop

The algo repeats steps 2 and 3 $K$ number of times before going back to step 1. That's what makes it 'fitted'; the nested loop. We will use 40 fitting steps $K$. This is epoch in code.

In [ ]:
nfq_results = []
best_agent, best_eval_score = None, float('-inf')
for seed in SEEDS:
  environment_settings = {
      'env_name': 'CartPole-v1',
      'gamma': 1.00,
      'max_minutes': 20,
      'max_episodes': 10000,
      'goal_mean_100_reward': 475
  }

  value_model_fn = lambda nS, nA: FCQ(nS, nA, hidden_dims=(512, 128))
  value_optimizer_fn = lambda net, lr: optim.RMSprop(net.parameters(), lr=lr)
  value_optimizer_lr = 5e-4

  training_strategy_fn = lambda: EGreedyStrategy(epsilon=0.5) # or 0.05
  evaluation_strategy_fn = lambda: GreedyStrategy()

  batch_size = 1024
  epochs = 40

  env_name, gamma, max_minutes, max_episodes, goal_mean_100_reward = environment_settings.values()
  agent = NFQ(value_model_fn,
              value_optimizer_fn,
              value_optimizer_lr,
              training_strategy_fn,
              evaluation_strategy_fn,
              batch_size,
              epochs)

  make_env_fn, make_env_kargs = get_make_env_fn(env_name=env_name)
  result, final_eval_score, training_time, wallcock_time = agent.train(
      make_env_fn, make_env_kargs, seed, gamma, max_minutes, max_episodes, goal_mean_100_reward
  )
  nfq_results.append(result)
  if final_eval_score > best_eval_score:
    best_eval_score = final_eval_score
    best_agent = agent

nfq_results = np.array(nfq_results)
_ = BEEP()
